In [ ]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd
import seaborn as sns

In [ ]:
sumPath = r'Y:\GECI_screening\wHaloCaMP'
dates = ['20230515_wHaloCaMP_raw','20230522_wHaloCaMP_raw']
df_curr = pd.DataFrame()
for date in dates:
    data = glob(os.path.join(sumPath,'combo & single mutants 2023',date)+'\\*.pkl')[0]
    df_temp = pd.read_pickle(data)
    df_curr = pd.concat([df_curr,df_temp],axis=0)
print(df_curr.shape)

In [ ]:
# df.shape

In [ ]:
df_curr = df_curr.dropna()
df_curr.shape

In [ ]:
# df_csv = df_csv.dropna()
 
# for c in df_csv.Construct.unique():
#     df_temp = df_csv[df_csv['Construct']==c]
#     if df_temp.Plate.unique().shape[0]!=1:
#         print(c,df_temp.Plate.unique())

# df_csv[df_csv['Construct']=='139.121']

In [ ]:
sns.set_style("whitegrid")
x = np.arange(1,1401)/206.23
Stim = ['001FP','003FP','010FP','160FP']
figPath = r'Z:\GECIScreenData\GECI_Imaging_Data\wHaloCaMP\summary\dFF_traces_fig'

for c in df_curr.Construct.unique():
    if c in ['139dot1','141dot1']:
        continue
    print(c)
    df_c = df_curr[df_curr.Construct==c]
    
    if df_c.Stim.unique().shape[0]!=4:
        print(c,'Stim file missing')
        continue
        
    for p in df_c.Plate.unique():
        df_plot = df_c[(df_c.Plate==p)&(df_c.Curated=='False')&(df_c['Filter']=='EX 660/13, DM 677, EM 719/60')]
        if len(df_plot) == 0:
            print('p',p)
            continue
            
#         if os.path.exists(os.path.join(figPath,df_plot.Date.unique()[0],p,c+'.png')):
#             print('file exists!')
#             continue
            
        fig = plt.figure(figsize=[12,4])
        plt.rc('font', size=13)
        i=1
        for stim in Stim:
            df_plot = df_c[(df_c.Plate==p)&(df_c['Stim']==stim)&(df_c.Curated=='False')&(df_c['Filter']=='EX 660/13, DM 677, EM 719/60')]
            
            ax = plt.subplot(1,4,i)

            mean = np.mean(df_plot.DFF.to_list(),axis=0)*100
            ax.plot(x,mean,color='k',alpha=0.6,label=c+' (%s)' %str(len(df_plot)))
            err = np.std(df_plot.DFF.to_list(),axis=0)*100/np.sqrt(len(df_plot))
            ax.fill_between(x,mean+err,mean-err,color = 'k',alpha=0.3)
            
            df_plot_control = df_curr[(df_curr['Plate']==p)&(df_curr['Construct']=='139dot1')&(df_curr['Stim']==stim)&(df_curr['Curated']=='False')&(df_curr['Filter']=='EX 660/13, DM 677, EM 719/60')]
            if len(df_plot_control)==0:
                df_plot_control = df_curr[(df_curr['Plate']==p)&(df_curr['Construct']=='141dot1')&(df_curr['Stim']==stim)&(df_curr['Curated']=='False')&(df_curr['Filter']=='EX 660/13, DM 677, EM 719/60')]
                if len(df_plot_control)==0:
                    print('no control', df_plot.Plate.unique()[0])
                    continue
            mean = np.mean(df_plot_control.DFF.to_list(),axis=0)*100
            ax.plot(x,mean,color='r',alpha=0.6,label=df_plot_control['Construct'].unique()[0]+' (%s)' %str(len(df_plot_control)))
            err = np.std(df_plot_control.DFF.to_list(),axis=0)*100/np.sqrt(len(df_plot))
            ax.fill_between(x,mean+err,mean-err,color='r',alpha=0.3)
            
            if i==1:
                ax.set_ylabel('\u0394F/F0 (%)',size=14)
                ax.legend(frameon=False)
                
            ax.set_title(Stim[i-1],size=14)
            ax.get_yaxis().set_label_coords(-0.3,0.5) # Set coords for y label so the figure body size remains the same
            ax.xaxis.set_label_coords(1,-0.05)               
            ax.set_xlabel('S')         
            i+=1
        plt.tight_layout()
        for d in df_plot.Date.unique():
            savePath = os.path.join(figPath,d,p)
            if not os.path.exists(savePath):
                os.makedirs(savePath)
            fig.savefig(savePath+'\\%s.png' %c,dpi=1200)
            plt.show()
            plt.close()

In [ ]:
# df

In [ ]:
# df_c.Filter.unique()

In [ ]:
# df_plot_control = df[(df['Plate']==df_plot.Plate.unique()[0])&(df['Construct_ID']=='139.1')&(df['Stim']==stim)&(df['Curated']=='False')]
# df_plot_control

In [ ]:
# x = np.arange(1,1401)/206.23
# ax = plt.subplot(111)
# mean = np.mean(df_plot_control.DFF.to_list(),axis=0)*100
# ax.plot(x,mean,color='k',alpha=0.6)
# err = np.std(df_plot_control.DFF.to_list(),axis=0)*100/np.sqrt(len(df_plot))
# ax.fill_between(x,mean+err,mean-err,color = 'k',alpha=0.3)

In [ ]:
# df_plot

#### Count number of plots (constructs)

In [ ]:
imgPath = r'Z:\GECIScreenData\GECI_Imaging_Data\wHaloCaMP\summary\dFF_traces_fig'
datePath = [d for d in os.listdir(imgPath) if os.path.isdir(os.path.join(imgPath,d))]

c = 0
for d in datePath:
    platePath = [p for p in os.listdir(os.path.join(imgPath,d)) if os.path.isdir(os.path.join(imgPath,d,p))]
    for p in platePath:
        img = glob(os.path.join(imgPath,d,p)+'\\*.png')
        c+=len(img)
c